In [1]:
# GET stock data

In [2]:
import json
import requests
import pandas as pd
import time
from fake_useragent import UserAgent
from datetime import datetime
from sqlalchemy import create_engine
from collections import Counter
import mysql.connector

In [3]:
ua = UserAgent()
headers = {"User-Agent":ua.random}
stock = "2330"
local_date = datetime.now().strftime('%Y%m%d')
timestamp = str(int(time.time()))
url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+local_date+"&stockNo="+stock+"&_=" +timestamp


In [4]:
res = requests.get(url,headers=headers)
json_stock = json.loads(res.content)

In [5]:
def transform_date(date):
        y, m, d = date.split('/')
        return str(int(y)+1911) + '/' + m  + '/' + d


In [6]:
result = []
for i in range(len(json_stock["data"])):
    Date = datetime.strptime(transform_date(json_stock["data"][i][0]),"%Y/%m/%d")
    Transaction_count = int(json_stock["data"][i][1].replace(",",""))
    Transaction_amount = int(json_stock["data"][i][2].replace(",",""))
    Opening_price = float(json_stock["data"][i][3])
    Highest_price = float(json_stock["data"][i][4])
    Lowest_price = float(json_stock["data"][i][5])
    Closing_price = float(json_stock["data"][i][6])

    if json_stock["data"][i][7] == "X0.00":
        Price_spread = 0.00
    else:
        Price_spread = float(json_stock["data"][i][7])

    Number_of_transactions = int(json_stock["data"][i][8].replace(",",""))

    dict_result = {
        "stock":stock,
        "Date":Date,
        "Transaction_count":Transaction_count,
        "Transaction_amount":Transaction_amount,
        "Opening_price":Opening_price,
        "Highest_price":Highest_price,
        "Lowest_price":Lowest_price,
        "Closing_price":Closing_price,
        "Price_spread":Price_spread,
        "Number_of_transactions":Number_of_transactions

    }
    
    result.append(dict_result)


In [13]:
pd.DataFrame(result)

,Closing_price,Date,Highest_price,Lowest_price,Number_of_transactions,Opening_price,Price_spread,Transaction_amount,Transaction_count,stock
0,295.5,2020-06-01,296.5,293.5,15502,294.0,3.5,11205227094,37936214,2330
1,296.5,2020-06-02,297.5,296.0,9743,296.0,1.0,7908476089,26663587,2330
2,301.0,2020-06-03,301.0,298.0,33062,300.0,4.5,20354971677,67894337,2330
3,306.0,2020-06-04,306.0,304.0,24482,305.0,5.0,14414983532,47225322,2330
4,311.5,2020-06-05,312.0,308.0,24096,308.5,5.5,13655618646,44077262,2330
5,318.0,2020-06-08,319.0,315.0,28581,316.0,6.5,16500836851,52042921,2330
6,319.0,2020-06-09,319.0,314.0,19981,316.5,1.0,11746874522,37068082,2330
7,322.5,2020-06-10,324.0,318.0,22670,319.0,3.5,13566733306,42207259,2330
8,320.5,2020-06-11,327.0,318.5,27061,325.5,-2.0,16340456218,50612255,2330
9,316.0,2020-06-12,317.5,312.5,25263,313.0,-4.5,14811252573,46970918,2330


In [7]:
def mysql_to_df():
    mydb = mysql.connector.connect(
            host="ntcpn108.ddns.net",
            user="adminer",
            passwd="Pn123456",
            database="Education"
    )

    mycursor = mydb.cursor()
    mycursor.execute("SELECT * FROM keyword")
    myresult = mycursor.fetchall()
    df = pd.DataFrame(myresult,columns=['id','Url', 'Key1', 'Count1', 'Key2', 'Count2',
                                        'Key3', 'Count3', 'Key4','Count4','Key5','Count5'])
    
    mycursor.close()
    mydb.close()
    
    return df